# Merge & compare

The purpose of this notebook is to merge data resulting from processing at different scales and to compare how the results differ.

In [1]:
import os
import glob
import pandas as pd
from collections import defaultdict

## Merge results

The input data contains column with species' name, result and extra column that is not used (".geo).

In [2]:
input_data_path = '../data/'
output_data_path = '../results/'

In [3]:
csv_paths = glob.glob(input_data_path + '*loss*.csv')
filenames = [os.path.splitext(os.path.basename(path))[0] for path in csv_paths]
scales = set([s[s.rfind('_') + 1:] for s in filenames])
print(f'Found following scale in data: {scales}')

Found following scale in data: {'120', '480', '240', '960', '1920'}


In [4]:
data_by_scale = {}

for scale in scales:
    all_files = glob.glob(input_data_path + '*' + scale + '.csv')
    loss_files = [path for path in all_files if 'loss' in path]
    df_all_years = []
    for path in loss_files:
        df = pd.read_csv(path)
        df = df.drop(labels='.geo', axis=1)
        year_idx_start = path.rfind('loss_') + len('loss_')
        year = path[year_idx_start: year_idx_start + 4] # 4 = number of digits in "year"
        df.columns = ['species', f'loss_{year}']
        df_all_years.append(df)
        
    df = pd.concat(df_all_years, axis=1)
    df = df.loc[:,~df.columns.duplicated()]
    df = df.reindex(sorted(df.columns), axis=1)
    
    treecover_path = (set(all_files) - set(loss_files)).pop()
    treecover_df = pd.read_csv(treecover_path)
    
    df.insert(0, 'treecover_2000', treecover_df['treecover2000'])
    df = df.set_index('species')
    df.to_csv(f'{output_data_path}large_sample_{scale}.csv')
    data_by_scale[scale] = df

## Compare results

In [10]:
reference_scale = '120'
compare_to = '1920'
(data_by_scale[compare_to] - data_by_scale[reference_scale]) / data_by_scale[reference_scale] * 100

,treecover_2000,loss_2001,loss_2002,loss_2003,loss_2004,loss_2005,loss_2006,loss_2007,loss_2008,loss_2009,loss_2010,loss_2011,loss_2012,loss_2013,loss_2014,loss_2015,loss_2016,loss_2017,loss_2018
species,,,,,,,,,,,,,,,,,,,
Accipiter_gentilis1,1.181886,-63.442686,-47.466255,-45.500957,-52.415392,-57.031300,-56.105563,-57.620287,-56.258500,-63.937731,-59.044061,-51.853734,-48.307255,-36.467918,-45.355070,-44.743057,-45.889179,-42.155387,-48.588069
Accipiter_gentilis3,1.623141,-89.649052,-83.134385,-91.837530,-91.232715,-92.297905,-91.726319,-91.444323,-90.443312,-94.732551,-95.440999,-95.270796,-92.234549,-91.285976,-91.433997,-93.064528,-87.293613,-86.403359,-93.559793
Accipiter_nisus1,1.494242,-97.198310,-97.945142,-94.366250,-96.582180,-95.446673,-97.125880,-95.430119,-97.226145,-96.089527,-90.769852,-67.489487,-95.135090,-96.452572,-95.816672,-97.314377,-92.782671,-82.780749,-93.657901
Accipiter_nisus2,1.150180,-55.787593,-44.636377,-35.825854,-48.642606,-76.298615,-58.060619,-50.817333,-49.412180,-59.966767,-71.261117,-48.352521,-40.160823,-29.594988,-31.571436,-52.779126,-30.148503,-31.585482,-34.888114
Accipiter_nisus3,1.762328,-98.774827,-96.486635,-96.329840,-95.409841,-94.463746,-93.596323,-92.614055,-93.541539,-92.634625,-94.647428,-95.172745,-93.279779,-94.377175,-92.524200,-93.445114,-92.090884,-93.371036,-95.259280
Bucephala_clangula2,1.135239,-57.280077,-40.962402,-39.287776,-45.544899,-49.371721,-49.584343,-51.676947,-50.521577,-58.050177,-51.793647,-46.613084,-42.365662,-29.483388,-37.981213,-36.482746,-37.434872,-35.524250,-41.131731
Bucephala_clangula3,1.338904,-78.684059,-75.227185,-75.573103,-79.405335,-83.844495,-79.744716,-74.229799,-75.391664,-82.046568,-83.126190,-81.205402,-79.119884,-81.191958,-81.014869,-79.900741,-78.728653,-73.725694,-78.521618
Coccyzus_americanus1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Coccyzus_americanus2,1.144296,-79.987975,-82.488596,-81.962770,-83.295445,-82.714747,-79.872922,-79.915729,-80.547603,-82.954005,-80.847474,-78.592283,-79.888182,-81.465371,-82.159966,-80.710075,-79.755660,-79.999464,-84.971059
